# Simple Workflow

In [ ]:
def fun1(input1):
    return input1 + "from fun1"

def fun2(input2):
    return input2 + " from fun2"

In [ ]:
from langgraph.graph import Graph

In [ ]:
workflow1 = Graph()

In [ ]:
workflow1.add_node("fun1", fun1)
workflow1.add_node("fun2", fun2)

workflow1.add_edge("fun1", "fun2")

In [ ]:
workflow1.set_entry_point("fun1")

In [ ]:
workflow1.set_finish_point("fun2")

In [ ]:
app1 = workflow1.compile()

In [ ]:
app1

In [ ]:
app1.invoke("Hello World, Welcome ")

# Workflow with LLM

In [ ]:
from langchain_groq.chat_models import ChatGroq
from langgraph.graph import Graph

In [ ]:
def model(input):
    llm = ChatGroq(model="gemma2-9b-it")
    return llm.invoke(input).content

In [ ]:
def upper_case(input):
    return input.upper()

In [ ]:
workflow2 = Graph()

In [ ]:
workflow2.add_node("model", model)
workflow2.add_node("upper_case", upper_case)

workflow2.add_edge("model", "upper_case")

workflow2.set_entry_point("model")
workflow2.set_finish_point("upper_case")

app2 = workflow2.compile()

In [ ]:
app2

In [ ]:
app2.invoke("who is the president of the India?")

# Workflow with RAG

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS 
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.prompts import PromptTemplate
from langchain_groq.chat_models import ChatGroq
from langgraph.graph import Graph


In [ ]:
path = r"arxiv_mcp.pdf"
loader = PyPDFLoader(path)
documents = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splitted_documents = text_splitter.split_documents(documents)

In [ ]:
emebeddings = OllamaEmbeddings(model="llama3.2")
db = FAISS.from_documents(splitted_documents, emebeddings)
retriever = db.as_retriever()

In [ ]:
prompt = PromptTemplate.from_template(
    """
    You are a helpful assistant. You will be provided with a question and a context. Answer the question based on the context.
    Context: {context}
    Question: {input}
    """
    )

In [ ]:
llm = ChatGroq(model="gemma2-9b-it")

In [ ]:
def rag_model(input):
    documents_chain = create_stuff_documents_chain(llm, prompt)
    rag_chain = create_retrieval_chain(retriever, documents_chain)
    response = rag_chain.invoke({"input":input})
    return response["answer"]

In [ ]:
def llm_model(input):
    llm = ChatGroq(model="gemma2-9b-it")
    return llm.invoke(input).content

In [ ]:
workflow3 = Graph()
workflow3.add_node("RAG_model", rag_model)
workflow3.add_node("LLM_model", llm_model)
workflow3.add_edge("RAG_model", "LLM_model")
workflow3.set_entry_point("RAG_model")
workflow3.set_finish_point("LLM_model")
app3 = workflow3.compile()


In [ ]:
app3

In [ ]:
res = app3.invoke("What is the main theme of the paper?")
print(res)

In [ ]:
from IPython.display import Image, display
try:
    display(Image(app3.get_graph().draw_mermaid_png()))
except Exception as e:
    # This requires some extra dependencies and is optional
    print(e)

In [ ]:
input = "What is the main theme of the paper?"
for output in app3.stream(input):
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

# Workflow with Conditional

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

In [ ]:
path = r"arxiv_mcp.pdf"
loader = PyPDFLoader(path)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splitted_documents = text_splitter.split_documents(documents)
emebeddings = OllamaEmbeddings(model="llama3.2")
db = FAISS.from_documents(splitted_documents, emebeddings)
retriever = db.as_retriever()

In [ ]:
from pydantic import BaseModel , Field
class TopicSelectionParser(BaseModel):
    Topic: str = Field(description='Selected Topic')
    Reasoning: str = Field(description='Reasoning behind topic selection')

In [ ]:
from langchain.output_parsers import PydanticOutputParser
parser = PydanticOutputParser(pydantic_object=TopicSelectionParser)

In [ ]:
from langchain_groq.chat_models import ChatGroq
llm = ChatGroq(model="gemma2-9b-it")

In [ ]:
from langchain.prompts import PromptTemplate
def agent(state):
    message=state["messages"]
    question=message[-1]
    print(question)
    
    template="""
    Your task is to classify the given user query into one of the following categories: [model context protocol(MCP), Not Related]. 
    Only respond with the category name and nothing else.
    User query: {question}
    {format_instructions}
    """
    
    prompt = PromptTemplate(template=template,
                            input_variables=[question],
                            partial_variables={"format_instructions" : parser.get_format_instructions()})
    
    chain =  prompt | llm | parser
    response = chain.invoke({"question":question,"format_instructions" : parser.get_format_instructions()})

    print(response)
    return {"messages": [response.Topic]}

In [ ]:
def router(state):
    print('-> Router ->')
    messages = state["messages"]
    last_message = messages[-1]
    print(last_message)
    if 'model context protocol'.lower() or 'mcp'.lower() in last_message:
        return 'RAG Call'
    else:
        return 'LLM Call'

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
def rag_model(state):
    print('-> Calling RAG ->')
    messages = state['messages']
    question = messages[0]
    print(question)
    template = """Answer the question based only on the following context:{context}
    Question: {question}
    """
    prompt = ChatPromptTemplate.from_template(template)
    print(prompt)

    retrieval_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
        )
    result = retrieval_chain.invoke(question)
    return  {"messages": [result]}

In [ ]:
def llm_model(state):
    print('-> Calling LLM ->')

    messages = state['messages']
    question = messages[0] ## Fetching the user question

    # Normal LLM call
    complete_query = "Anwer the follow question with your knowledge of the real world. Following is the user question: " + question
    response = llm.invoke(complete_query)
    return {"messages": [response.content]}

In [ ]:
from typing import TypedDict, Sequence, Annotated
import operator
from langchain_core.messages import BaseMessage
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

In [ ]:
from langgraph.graph import StateGraph,END
workflow4 = StateGraph(AgentState) ### StateGraph with AgentState

workflow4.add_node("agent", agent)
workflow4.add_node("RAG", rag_model)
workflow4.add_node("LLM", llm_model)


workflow4.set_entry_point("agent")
workflow4.add_conditional_edges(
    "agent",
    router,
    {
        "RAG Call": "RAG",
        "LLM Call": "LLM",
    }
)
workflow4.add_edge("RAG",END)
workflow4.add_edge("LLM",END)
app4=workflow4.compile()

In [ ]:
from IPython.display import Image, display
try:
    display(Image(app4.get_graph().draw_mermaid_png()))
except Exception as e:
    # This requires some extra dependencies and is optional
    print(e)

In [ ]:
inputs = {"messages": ["Tell me about the model context protocol"]}

In [ ]:
output = app4.invoke(inputs)

In [ ]:
print(output.get("messages")[-1])  # Print the last message in the output